# Track Predictions and Custom Metrics Locally

This notebook shows a full example how to set up a decorated prediction function and test it locally, based on the **logged** model from the previous lab.

- ✅ Load the MLflow model from logged artifacts based on accuracy.
- ✅ Set up a decorated prediction function to log metrics during inference.
- ✅ Test the metrics store locally in "development" mode.

In [84]:
import mlflow
import pandas as pd

import cml.metrics_v1 as metrics
import cml.models_v1 as models

# Configuration for experiment and model
EXPERIMENT_NAME = "sklearn_experiment_lab_2"
MODEL_ARTIFACT_LOCATION = "logistic_regression"

# Load the best performing model from MLflow based on accuracy
run_results = mlflow.search_runs(experiment_names=[EXPERIMENT_NAME])
best_run_artifact_uri = run_results.loc[run_results["metrics.accuracy"].idxmax(), "artifact_uri"]
model = mlflow.pyfunc.load_model(f"{best_run_artifact_uri}/{MODEL_ARTIFACT_LOCATION}")
print(f"Loaded best model from artifacts location: {best_run_artifact_uri}")

Loaded best model from artifacts location: /home/cdsw/.experiments/6yxz-cbhb-5jsi-afk2/tag1-yyl1-367x-rjfl/artifacts


In [85]:
# The model_metrics decorator equips the predict function to
# call track_metrics. It also changes the return type. If the
# raw predict function returns a value "result", the wrapped
# function will return eg
# {
#   "uuid": "612a0f17-33ad-4c41-8944-df15183ac5bd",
#   "prediction": "result"
# }
# The UUID can be used to query the stored metrics for this
# prediction later.
@models.cml_model(metrics=True)
def predict(inputs: pd.DataFrame):
    """
    Make predictions with the model while tracking metrics.
    
    Args:
        inputs: DataFrame containing input features
        
    Returns:
        Dictionary containing:
        - prediction: The model's prediction
        - uuid: Unique identifier for tracking this prediction's metrics
    """
    # Track the input features for monitoring
    metrics.track_metric("input", inputs["inputs"][0])

    # Run model inference
    result = model.predict(inputs)

    # Track the prediction output
    metrics.track_metric("output", str(result[0]))

    return result


Not running in a model replica, so using a local development
version of the model metrics service. Please use the following
CRN's to consume metrics:
   model_crn: "crn:cdp:ml:::workspace:dev/model" (cml.metrics.dev_model_crn)
   model_build_crn: "crn:cdp:ml:::workspace:dev/model-build" (cml.metrics.dev_model_build_crn)
   model_deployment_crn: "crn:cdp:ml:::workspace:dev/model-deployment" (cml.metrics.dev_model_deployment_crn)



In [88]:
# Define example input and invoke predict function with metrics
example_input = pd.DataFrame({"inputs": [[4.6, 3.6, 1.0, 0.2]]})
prediction = predict(example_input)

prediction

{'prediction': array([0]),
 'model_deployment_crn': 'crn:cdp:ml:::workspace:dev/model-deployment',
 'uuid': '5f94cb41-eb5b-4b8b-a750-67874f7d4aad'}